# Julia OpenCL

In this notebook we will look at implementing a diffusion equation in Julia, first in native terms and then using Julia’s OpenCL bindings.

## A note on multi-dimensional arrays

Julia’s multi-dimensional arrays are stored in column-major order (and indexed starting at 1) just like Fortran, whereas array handling in OpenCL are stored in row-major order


row-major, column-major …

\begin{align}
A = \begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \end{bmatrix}
\end{align}

In [ ]:
m = ones(3, 4, 5)
m[:] = 1:3*4*5
m

In [ ]:
using PyCall

In [ ]:
@pyimport numpy as np

In [ ]:
m = np.reshape(np.array(py"range"(3 * 4 * 5)), (3, 4, 5))

In [ ]:
m[1, 1, 2], m[1, 2, 1], m[2, 1, 1]

In [ ]:
np.array(py"range"(3 * 4 * 5), shape=(3, 4, 5))

In [ ]:
Pkg.add("PyCall")
Pkg.update()

In [ ]:
m[2, 1, 1]

In [ ]:
versioninfo()

In [ ]:
using OpenCL
#using Gadfly

In [ ]:
diff_kernel = "
inline int xy_to_i(int x, int y, int col_size, int mesh_size) {
    return (x % col_size) + (y % col_size) * col_size; 
}

__kernel void diff(__global const float *a,
__global float *diff,
int col_size,
int mesh_size)
    {
      int gid = get_global_id(0);
    int x_idx = gid % col_size;
    int y_idx = gid / col_size;
    
    diff[gid] = - 4. * a[gid]
    + a[xy_to_i(x_idx - 1, y_idx, col_size, mesh_size)]
    + a[xy_to_i(x_idx + 1, y_idx, col_size, mesh_size)]
    + a[xy_to_i(x_idx, y_idx - 1, col_size, mesh_size)]
    + a[xy_to_i(x_idx, y_idx + 1, col_size, mesh_size)];
    }
";

In [ ]:
diff_kernel_2d = "
__kernel void diff2d(__global const float *a,
__global float *diff)
    {
      int gidx = get_global_id(0);
      int gidy = get_global_id(1);
      int maxx = get_global_size(0);
      int maxy = get_global_size(1);

   float (*b)[maxy] = (float(*)[maxy]) a;
   float (*d)[maxx] = (float(*)[maxx]) diff;

    d[gidx][gidy] = - 4. * b[gidx][gidy]
    + b[(gidx - 1) % maxx][gidy]
    + b[(gidx + 1) % maxx][gidy]
    + b[gidx][((gidy - 1) % maxy)]
    + b[gidx][((gidy + 1) % maxy)];
    /*
    diff[gidx + gidy * maxx] = - 4. * a[gidx + gidy * maxx]
    + a[(gidx - 1) % maxx + gidy * maxx]
    + a[(gidx + 1) % maxx + gidy * maxx]
    + a[gidx + ((gidy - 1) % maxy) * maxx]
    + a[gidx + ((gidy + 1) % maxy) * maxx];*/
    }
";

In [ ]:
# TODO 2D kernel


In [ ]:
sum_kernel = "
   __kernel void sum(__global float *a,
__global const float *diff,
float step_size)
    {
      int gid = get_global_id(0);
    a[gid] += step_size * diff[gid];
    }
";

In [8]:
function initial_cond(s=5000, t=s)
    a = zeros(Float32, (s, t))
    a[Int(size(a)[1] / 2), Int(size(a)[2] / 2)] = 10
    return a
end

initial_cond (generic function with 3 methods)

In [ ]:
function do_openCL(input)
#    device, ctx, queue = cl.create_compute_context()
#    print(device, ctx, queue)
    ctx    = cl.Context(cl.devices()[2])
    device = cl.devices(ctx)
    queue  = cl.CmdQueue(ctx)
    print(ctx, device, queue)

    input_buff = cl.Buffer(Float32, ctx, (:rw, :copy), hostbuf=input)
    diff_buff = cl.Buffer(Float32, ctx, :rw, length(input))

    diff_p = cl.Program(ctx, source=diff_kernel) |> cl.build!
    diff_k = cl.Kernel(diff_p, "diff")

    sum_p = cl.Program(ctx, source=sum_kernel) |> cl.build!
    sum_k = cl.Kernel(sum_p, "sum")

    step_size = 0.001

    for i in 1:2
        queue(diff_k, length(input), nothing, input_buff, diff_buff, size(input)[1], length(input))
        queue(sum_k, length(input), nothing, input_buff, diff_buff, Float32(step_size))
    end
    r = cl.read(queue, input_buff);
    r = reshape(r, size(input))
    return r
end



In [ ]:
function do_openCL2d(input)
    device, ctx, queue = cl.create_compute_context()
    print(device, ctx, queue)
    input_buff = cl.Buffer(Float32, ctx, (:rw, :copy), hostbuf=input)
    diff_buff = cl.Buffer(Float32, ctx, :rw, length(input))

    diff_p = cl.Program(ctx, source=diff_kernel_2d) |> cl.build!
    diff_k = cl.Kernel(diff_p, "diff2d")

    sum_p = cl.Program(ctx, source=sum_kernel) |> cl.build!
    sum_k = cl.Kernel(sum_p, "sum")

    step_size = 0.001

    for i in 1:2
        queue(diff_k, size(input), nothing, input_buff, diff_buff)
        queue(sum_k, length(input), nothing, input_buff, diff_buff, Float32(step_size))
    end
    r = cl.read(queue, input_buff);
    r = reshape(r, size(input))
    return r
end



In [ ]:
res = do_openCL2d(initial_cond(6, 10))

In [ ]:
maximum(res[:])

In [ ]:
device, ctx, queue = cl.create_compute_context()
methods(queue)

In [ ]:
@time do_openCL(initial_cond(15000))

In [ ]:
cl.devices(cl.create_some_context())

In [ ]:
cl.cl_device_info(3)

In [ ]:
z = initial_cond(20000);

In [ ]:
whos(r"z")

In [ ]:
Pkg.add("GR")

In [ ]:
using GR
inline("mov")
x = [i for i in linspace(0, 2*pi, 100)]
dt, nsteps = 0.03, 30
for n = 1:nsteps
    GR.plot(x, sin(x - dt*n), "d")
end
GR.show()

In [ ]:
using Plots
Plots.plotlyjs()
x = linspace(0, 2*pi, 100) |> collect 
dt, nsteps = 0.03, 100
for n = 1:nsteps
    #IJulia.clear_output(true)
    Plots.plot(x, sin(x - n*dt))  |> display
end

In [ ]:
using GR
inline()
x = linspace(0, 2*pi, 100) |> collect 
dt, nsteps = 0.03, 100
for n = 1:nsteps
    IJulia.clear_output(true)
    GR.plot(x, sin(x - n*dt))  |> display
end

In [ ]:
Pkg.add("PlotlyJS")

In [ ]:
Pkg.add("Plots")

In [ ]:
cl.devices()

In [ ]:
cl.Context(cl.devices()[3])

In [9]:
function diff_step(input, output)
    maxX, maxY = size(input)
    @sync @parallel for iter in eachindex(input)
        x, y = ind2sub(input, iter)
        output[iter] = - 4. * input[iter] + input[x, y - 1 == 0 ? maxY : y - 1] + 
        input[x, y + 1 > maxY ? 1 : y + 1] +
        input[x - 1 == 0 ? maxX : x - 1, y] +
        input[x + 1 > maxX ? 1 : x + 1, y]
    end
end

diff_step (generic function with 1 method)

In [10]:
function sum_step(input, diff, step_size)
    @sync @parallel for iter in eachindex(input)
        input[iter] += (step_size * diff[iter])
    end
end
    

sum_step (generic function with 1 method)

In [11]:
function do_naive(input)
    step_size = 0.001

    diff = similar(input)
    for i in 1:10
        diff_step(input, diff)
        sum_step(input, diff, step_size)
    end
    return input
end

do_naive (generic function with 1 method)

In [17]:
addprocs()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [20]:
@time res = do_naive(initial_cond(500, 5000));

  4.998452 seconds (60.15 k allocations: 20.945 MB, 1.89% gc time)


In [ ]:
maximum(res)

In [ ]:
a = [1, 3, 5]
c = [2, 3, 5]
b = similar(a)
@time b += a * 5

In [ ]:
cl.devices()

In [ ]:
Pkg.add('PyPlot')

In [ ]:
initial_cond()[1,2]

In [1]:
stencil = [0 1 0; 1 -4 1; 0 1 0]

3×3 Array{Int64,2}:
 0   1  0
 1  -4  1
 0   1  0